# Procesamiento del lenguaje natural - 2025 - B4 - Desafio 3
**Inteligencia Artificial - CEIA - FIUBA**

## Autor

- **Mendoza Dante**.
- **SIU: e2206**.

**Nota:** Tomé como base el código compartido por los docentes. Como la idea era pasar de Keras a PyTorch, utilicé IA para consultar errores y adaptación de código. Las que utilicé fueron Copilot y ChatGPT. Elegí un ebook encontrado en el sitio: https://www.textos.info/alejandro-dumas/el-conde-de-montecristo/ebook

## Modelo de lenguaje con tokenización por caracteres
### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

Conclusiones

ALGO...

In [ ]:
################################################################################
# Bibliotecas y entorno CPU o GPU
################################################################################
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import math
from tqdm import tqdm
import random
import re
import requests
from bs4 import BeautifulSoup
import gradio as gr

# CPU o GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
# ==============================
# Variables globales
# ==============================
EPOCHS = 10
PATIENCE = 5
MAX_LEN = 10 # Caracteres
TEMPERATURE = 0.8

In [ ]:
################################################################################
# Descarga del texto
################################################################################
url = "https://www.textos.info/alejandro-dumas/el-conde-de-montecristo/ebook"
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")
paragraphs = soup.find_all("p")
text = " ".join(p.text for p in paragraphs)

# limpieza básica: minúsculas y caracteres permitidos
text = text.lower()
text = re.sub(r"[^a-záéíóúüñ0-9,.!?;:()\s-]", " ", text)

print("Longitud del corpus:", len(text))
print("Ejemplo:", text[:500])

Longitud del corpus: 2573547
Ejemplo:  el 24 de febrero de 1815, el vigía de nuestra señora de la guarda dio
 la señal de que se hallaba a la vista el bergantín el faraón procedente
 de esmirna, trieste y nápoles. como suele hacerse en tales casos, salió
 inmediatamente en su busca un práctico, que pasó por delante del 
castillo de if y subió a bordo del buque entre la isla de rión y el cabo
 mongión. en un instante, y también como de costum bre, se llenó de 
curiosos la plataforma del castillo de san juan, por que en marsella


In [ ]:
################################################################################
# Tokenización
################################################################################
chars_vocab = sorted(list(set(text)))
vocab_size = len(chars_vocab)

char2idx = {c:i for i,c in enumerate(chars_vocab)}
idx2char = {i:c for c,i in char2idx.items()}

# tokenizo
tokenized_text = [char2idx[c] for c in text]

In [ ]:
################################################################################
# Preparo el dataset
################################################################################
max_context_size = 100
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))

train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]

train_seqs = [train_text[i:i+max_context_size] for i in range(len(train_text)-max_context_size)]
val_seqs = [val_text[i*max_context_size:(i+1)*max_context_size] for i in range(num_val)]

X_train = np.array(train_seqs[:-1])
y_train = np.array(train_seqs[1:])

X_val = np.array(val_seqs[:-1])
y_val = np.array(val_seqs[1:])

In [ ]:
################################################################################
# Adapto a PyTorch
################################################################################
class CharDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 256
train_loader = DataLoader(CharDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(CharDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

In [ ]:
################################################################################
# Modelo RNN con Embeddings
################################################################################
class CharRNN(nn.Module):
    def __init__(self, model_type, vocab_size, emb_size=64, hidden_size=200, n_layers=1, dropout=0.1):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.model_type = model_type.lower()
        if self.model_type == "simplernn":
            self.rnn = nn.RNN(emb_size, hidden_size, num_layers=n_layers, batch_first=True, nonlinearity='tanh', dropout=dropout if n_layers>1 else 0)
        elif self.model_type == "lstm":
            self.rnn = nn.LSTM(emb_size, hidden_size, num_layers=n_layers, batch_first=True, dropout=dropout if n_layers>1 else 0)
        elif self.model_type == "gru":
            self.rnn = nn.GRU(emb_size, hidden_size, num_layers=n_layers, batch_first=True, dropout=dropout if n_layers>1 else 0)
        else:
            raise ValueError("model_type debe ser simplernn|lstm|gru")
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        emb = self.emb(x)
        out, hidden = self.rnn(emb, hidden)
        logits = self.fc(out)
        return logits, hidden

    def init_hidden(self, batch_size):
        num_layers = self.rnn.num_layers
        hidden_size = self.rnn.hidden_size
        if self.model_type == "lstm":
            h0 = torch.zeros(num_layers, batch_size, hidden_size).to(device)
            c0 = torch.zeros(num_layers, batch_size, hidden_size).to(device)
            return (h0, c0)
        else:
            h0 = torch.zeros(num_layers, batch_size, hidden_size).to(device)
            return h0

In [ ]:
################################################################################
# Loop de entrenamiento con perplejidad
################################################################################
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    total_tokens = 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            logits, _ = model(x)
            B,T,V = logits.size()
            loss = criterion(logits.view(B*T,V), y.view(B*T))
            total_loss += loss.item()*(B*T)
            total_tokens += B*T
    avg_loss = total_loss / total_tokens
    return avg_loss, math.exp(avg_loss)

def train(model, train_loader, val_loader, epochs=EPOCHS, lr=1e-3, clip=5.0, patience=PATIENCE):
    model.to(device)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    best_ppl = float('inf')
    patience_counter = 0
    best_state = None

    for epoch in range(1, epochs+1):
        model.train()
        total_loss = 0
        total_tokens = 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits, _ = model(x)
            B,T,V = logits.size()
            loss = criterion(logits.view(B*T,V), y.view(B*T))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            total_loss += loss.item()*(B*T)
            total_tokens += B*T
        train_loss = total_loss / total_tokens

        val_loss, val_ppl = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch} | train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | val_ppl {val_ppl:.4f}")

        if val_ppl < best_ppl:
            best_ppl = val_ppl
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping por perplejidad")
                break
    model.load_state_dict(best_state)
    return model

In [ ]:
################################################################################
# Funciones de generación
################################################################################
def greedy_generate(model, seed, stoi, itos, max_len=MAX_LEN):
    model.eval()
    idxs = [stoi[c] for c in seed if c in stoi]
    input_seq = torch.tensor(idxs, device=device).unsqueeze(0)
    hidden = None
    generated = idxs.copy()
    with torch.no_grad():
        for _ in range(max_len):
            logits, hidden = model(input_seq, hidden)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            topi = torch.argmax(probs, dim=-1).item()
            generated.append(topi)
            input_seq = torch.tensor([[topi]], device=device)
    return "".join([itos[i] for i in generated])

def stochastic_generate(model, seed, stoi, itos, max_len=MAX_LEN, temperature=TEMPERATURE):
    model.eval()
    idxs = [stoi[c] for c in seed if c in stoi]
    input_seq = torch.tensor(idxs, device=device).unsqueeze(0)
    hidden = None
    generated = idxs.copy()
    with torch.no_grad():
        for _ in range(max_len):
            logits, hidden = model(input_seq, hidden)
            probs = F.softmax(logits[:, -1, :].squeeze(0)/temperature, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1).item()
            generated.append(next_idx)
            input_seq = torch.tensor([[next_idx]], device=device)
    return "".join([itos[i] for i in generated])

In [ ]:
################################################################################
# Interfaz Gradio
################################################################################
def model_response(ingresar_texto):
    output_text = greedy_generate(model, ingresar_texto, char2idx, idx2char, max_len=MAX_LEN)
    return output_text

iface = gr.Interface(fn=model_response, inputs="text", outputs="text")

In [ ]:
################################################################################
# Entreno el modelo
################################################################################
model_type = "lstm"  # simplernn | lstm | gru
model = CharRNN(model_type, vocab_size=vocab_size, emb_size=64, hidden_size=200, n_layers=2, dropout=0.1)
model = train(model, train_loader, val_loader, epochs=EPOCHS, lr=1e-3, clip=5.0, patience=PATIENCE)

Epoch 1: 100%|██████████| 9048/9048 [06:06<00:00, 24.66it/s]


Epoch 1 | train_loss 1.2482 | val_loss 7.9340 | val_ppl 2790.4362


Epoch 2: 100%|██████████| 9048/9048 [06:07<00:00, 24.62it/s]


Epoch 2 | train_loss 1.0852 | val_loss 8.3614 | val_ppl 4278.7209


Epoch 3: 100%|██████████| 9048/9048 [06:07<00:00, 24.60it/s]


...

Epoch 4 | train_loss 1.0448 | val_loss 8.6300 | val_ppl 5596.9380


Epoch 5: 100%|██████████| 9048/9048 [06:07<00:00, 24.63it/s]


Epoch 5 | train_loss 1.0374 | val_loss 8.8186 | val_ppl 6758.8649


Epoch 6: 100%|██████████| 9048/9048 [06:07<00:00, 24.63it/s]

Epoch 6 | train_loss 1.0326 | val_loss 8.9481 | val_ppl 7693.2326
Early stopping por perplejidad


In [ ]:
################################################################################
# Guardado del modelo
################################################################################
torch.save(model, "char_rnn_full.pth") # se que se guarda en la memoria de la sesion pero sirve para descargarlo

In [ ]:
################################################################################
# Carga del modelo
################################################################################
model = torch.load("char_rnn_full.pth")
model.to(device)
model.eval()

In [ ]:
################################################################################
# Lanzo Gradio
################################################################################
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7d37daa90df79c8368.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7d37daa90df79c8368.gradio.live


In [ ]:
################################################################################
# Beam Search (determinístico y estocástico)
################################################################################
from scipy.special import softmax

def beam_search(model, seed, stoi, itos, max_len=MAX_LEN, beam_width=3, temperature=TEMPERATURE, stochastic=False):
    model.eval()
    idxs = [stoi[c] for c in seed if c in stoi]
    sequences = [(idxs, 0.0, None)]

    with torch.no_grad():
        # inicializo hidden con todo el contexto
        input_seq = torch.tensor(idxs, device=device).unsqueeze(0)
        _, hidden = model(input_seq, None)

        for _ in range(max_len):
            all_candidates = []
            for seq, log_prob, hid in sequences:
                last_token = torch.tensor([[seq[-1]]], device=device)
                logits, new_hid = model(last_token, hidden if hid is None else hid)
                probs = F.softmax(logits[:, -1, :].squeeze(0), dim=-1).cpu().numpy()

                if stochastic:
                    # muestreo estocástico con temperatura
                    probs_temp = softmax(np.log(probs+1e-20)/temperature)
                    top_idxs = np.random.choice(len(probs_temp), beam_width, p=probs_temp)
                else:
                    # determinístico: top k
                    top_idxs = probs.argsort()[-beam_width:][::-1]

                for idx in top_idxs:
                    candidate = (seq + [idx], log_prob + math.log(probs[idx]+1e-20), new_hid)
                    all_candidates.append(candidate)

            # me quedo con los beam_width mejores
            sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]

        # devuelvo la secuencia con mayor log_prob
        best_seq = sequences[0][0]
        return "".join([itos[i] for i in best_seq])

In [ ]:
################################################################################
# Utilizo Beam Search (determinístico y estocástico)
################################################################################
seed_text = "el cond"
print("Greedy:", greedy_generate(model, seed_text, char2idx, idx2char, max_len=MAX_LEN))
print("Beam search det:", beam_search(model, seed_text, char2idx, idx2char, max_len=MAX_LEN, beam_width=5, stochastic=False))
print(f"Beam search sto (T={TEMPERATURE}):", beam_search(model, seed_text, char2idx, idx2char, max_len=MAX_LEN, beam_width=5, stochastic=True, temperature=TEMPERATURE))

Greedy: el conde de monte
Beam search det: el conde de morce
Beam search sto (T=0.8): el condo de monte
